In [1]:
from scipy.spatial import Voronoi, voronoi_plot_2d
import numpy as np 
%matplotlib inline

In [2]:
import random
def random_color(as_str=True, alpha=0.5):
    rgb = [random.randint(0,255),
           random.randint(0,255),
           random.randint(0,255)]
    if as_str:
        return "rgba"+str(tuple(rgb+[alpha]))
    else:
        return list(np.array(rgb)/255) + [alpha]
    

In [3]:

def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    Source
    -------
    Copied from https://gist.github.com/pv/8036995
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max() * 2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

def voronoi_polygons(n=50):
    random_seeds = np.random.rand(n, 2)
    vor = Voronoi(random_seeds)
    regions, vertices = voronoi_finite_polygons_2d(vor)
    polygons = []
    for reg in regions:
        polygon = vertices[reg]
        polygons.append(polygon)
    return polygons

In [4]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import folium 
from folium import vector_layers 

In [5]:
import geopandas
import pandas as pd

hospitals = pd.DataFrame(geopandas.read_file("../data/Hospitals.geojson"))
hospitals = hospitals[hospitals["HELIPAD"]=="Y"]

def is_level_one(x):
    """True if x is at least a level one trauma center, else False"""
    isin = ["LEVEL I,","LEVEL I ADULT","I, I "]
    iseq = ["1","I","LEVEL 1","LEVEL I","PARC"]
    return any([key in x for key in isin]) or any([key == x for key in iseq])

# 
hospitals = hospitals[hospitals["TRAUMA"].apply(is_level_one)]
hospitals = hospitals[hospitals["STATE"]!="HI"] 
hospitals = hospitals[list(hospitals["TYPE"]!="CHILDREN")]

hospitals = hospitals[~hospitals["NAME"].isin(["PRIMARY CHILDREN'S MEDICAL CENTER"])]
hospitals = hospitals[~(hospitals["BEDS"]<=0).values]

hospitals = hospitals.reset_index(drop=True)
hospitals = hospitals[["LONGITUDE", 'LATITUDE',"BEDS","STATE"]]
hospitals.columns = ["lon","lat","beds","state"] 
hospitals[["lon","lat","beds"]] = hospitals[["lon","lat","beds"]].astype(float)

hospitals.head(5)

,lon,lat,beds,state
0,-71.073200,42.334700,368.0,MA
1,-72.603200,42.121700,724.0,MA
2,-86.802035,33.505603,1157.0,AL
3,-86.580806,34.721433,881.0,AL
4,-80.838902,35.203900,1023.0,NC


In [6]:
# Add Voronoi cell polygons 
def calc_polygons(df):
    vor = Voronoi(df[["lon","lat"]].values)
    regions, vertices = voronoi_finite_polygons_2d(vor)
    polygons = []
    for reg in regions:
        polygon = vertices[reg]
        polygons.append(polygon)
    return polygons
hospitals["polygons"] = calc_polygons(hospitals)
hospitals.head(5)

,lon,lat,beds,state,polygons
0,-71.073200,42.334700,368.0,MA,"[[-71.08943244172721, 42.351191631639416], [-7..."
1,-72.603200,42.121700,724.0,MA,"[[-73.32922615901754, 42.08013042290189], [-72..."
2,-86.802035,33.505603,1157.0,AL,"[[-85.61492935793834, 33.854880978226916], [-8..."
3,-86.580806,34.721433,881.0,AL,"[[-85.69360114455075, 33.93195704888111], [-86..."
4,-80.838902,35.203900,1023.0,NC,"[[-81.55185615205659, 35.83970559543777], [-81..."


In [7]:
import matplotlib as mpl
import matplotlib.cm as cm

def nums_to_color(series, cmap=cm.coolwarm_r, alpha=0.5):
    """
    See https://matplotlib.org/examples/color/colormaps_reference.html 
    for colormap names. 
    """
    
    norm = mpl.colors.Normalize(vmin=series.min(), 
                                vmax=series.max())
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    m_arr = m.to_rgba(series).reshape(len(series),4) * 255
    m_arr[:,3] = np.repeat(alpha, len(series))
    return list(m_arr)

hospitals["colors"] = nums_to_color(hospitals["beds"], alpha=0.9)
hospitals.head(5)

,lon,lat,beds,state,polygons,colors
0,-71.073200,42.334700,368.0,MA,"[[-71.08943244172721, 42.351191631639416], [-7...","[237.616980633, 132.366808416, 103.652300193, ..."
1,-72.603200,42.121700,724.0,MA,"[[-73.32922615901754, 42.08013042290189], [-72...","[236.018672865, 210.506911008, 196.639758492, ..."
2,-86.802035,33.505603,1157.0,AL,"[[-85.61492935793834, 33.854880978226916], [-8...","[152.433077751, 185.46907984499998, 254.943066..."
3,-86.580806,34.721433,881.0,AL,"[[-85.69360114455075, 33.93195704888111], [-86...","[210.8300007, 218.842365662, 231.27292438, 0.9]"
4,-80.838902,35.203900,1023.0,NC,"[[-81.55185615205659, 35.83970559543777], [-81...","[182.032385294, 206.25857104099998, 249.743553..."


In [8]:
hospitals.sort_values("beds")

,lon,lat,beds,state,polygons,colors
20,-97.332278,37.699899,58.0,KS,"[[-100.56519074889864, 40.96614211357133], [-1...","[179.94665529, 3.9668208, 38.30936706, 0.9]"
113,-80.838718,35.201885,70.0,NC,"[[-81.30179744900471, 34.675850910127465], [-7...","[182.945808954, 13.034974848, 40.4778349639999..."
67,-93.279677,37.144946,86.0,MO,"[[-94.93303276884328, 37.376049925172865], [-9...","[185.94496261799998, 22.103128896, 42.64630286..."
41,-104.754733,38.966846,88.0,CO,"[[-100.64655710451967, 40.658909583226], [-101...","[187.44453945, 26.63720592, 43.73053682, 0.9]"
122,-86.802892,33.504006,106.0,AL,"[[-85.43898649672275, 32.13076060512463], [-85...","[191.939504981, 40.181784807999996, 46.9857668..."


In [9]:
from folium.features import DivIcon

def plot_map(df, center=(39.8283, -98.5795), show_nums=False, show_seeds=True):
    m = folium.Map(location=[*center],
                   width=750, height=500, 
                   zoom_start=4,
                   api_key='6NbtVc32EkZBkf8eXLAE')
    
    for lat, lon, color, poly, beds in df[["lat","lon","colors","polygons","beds"]].values:
        points = to_convex(np.flip(poly).tolist())
        vlayer = vector_layers.Polygon(points, 
                                       fill=True, 
                                       color="black",
                                       fill_color="rgba({}, {}, {}, {})".format(*color),
                                       weight=1)
        m.add_child(vlayer)

        if show_seeds:
            clayer = vector_layers.Circle([lat,lon], 2, color="black")
            m.add_child(clayer)
        
        if show_nums:
            folium.Marker((lat, lon), icon=DivIcon(
            icon_size=(.1,.1),
            icon_anchor=(6,19),
            html='<div style="font-size: 5pt; color : black">%s</div>'%str(int(beds)),
            )).add_to(m)
        
    return m

plot_map(hospitals)

NameError: name 'to_convex' is not defined

In [19]:
import math
def to_convex(points):
    # compute centroid
    cent = (sum([p[0] for p in points])/len(points),
            sum([p[1] for p in points])/len(points))
    # sort by polar angle
    points.sort(key=lambda p: math.atan2(p[1] - cent[1],
                                         p[0] - cent[0]))
    return points

In [ ]:
hosps = hospitals.copy()[["lat","lon","beds"]]

In [ ]:
from sklearn.cluster import DBSCAN

def cluster(df, eps=0.5):
    # Get data
    df = df.copy()
    X = df[["lon","lat"]].values
    
    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=1).fit(X)
    
    # Temporarily add cluster labels to df
    df["cluster"] = db.labels_
    clustered_data = []
    for cluster in set(db.labels_):
        values = df[df["cluster"]==cluster].values
        beds = values[:,2].sum()
        center = values[:,0:2].mean(axis=0)
        clustered_data.append({"lat":center[0],
                               "lon":center[1],
                               "beds":beds})
    return pd.DataFrame(clustered_data)

clustered_hospitals = cluster( hosps , eps=0.25)

In [ ]:

clustered_hospitals["colors"] = nums_to_color(clustered_hospitals["beds"], alpha=0.8)
clustered_hospitals["polygons"] = calc_polygons(clustered_hospitals)

plot_map(clustered_hospitals, show_nums=False)

## m.save('m.html')


In [ ]:
from folium.plugins import HeatMap

center=(39.8283, -98.5795)
m = folium.Map(location=[*center],
                   width=750, height=500, 
                   zoom_start=4,
                   api_key='6NbtVc32EkZBkf8eXLAE')
HeatMap(data=hospitals[["lat","lon","beds"]].values, radius=8, max_zoom=13).add_to(m)
m

In [ ]:
m = folium.Map(location=center, zoom_start=4)
 
# Add the color for the chloropleth:
m.choropleth(
 geo_data="../data/us-states.json",
 name='choropleth',
 data=hospitals,
 columns=['state', 'beds'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2
)
m

In [17]:
m